In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/Users/yinchiahuang/Library/Mobile Documents/com~apple~CloudDocs/CS677/Assignments/yinchia@bu.edu_hw1/HMC_Honda Motor Co., Ltd.csv')

### Momentum ( moving average)

In [3]:
def get_ma_df(df,win_st,win_lt):
    ma_df = df.copy()
    ma_df.rename(columns={'Adj Close': 'Price'}, inplace = True)
    ma_df = ma_df[['Date', 'Price']]
    ma_df['ma_st'] = ma_df['Price'].rolling(win_st).mean()
    ma_df['ma_lt'] = ma_df['Price'].rolling(win_lt).mean()
    #ma_df = ma_df.tail(len(ma_df)-win_lt)
    ma_df['ma_buy'] = ma_df['ma_st'] > ma_df['ma_lt']
    ma_df['ma_sell'] = ma_df['ma_st'] < ma_df['ma_lt']
    ma_df['ma_transactions'] = np.where(ma_df['ma_buy'] != ma_df['ma_buy'].shift(+1),1,0)
    ma_df
    columnsTitles = ['Date','Price','ma_buy','ma_sell']
    ma_df = ma_df[columnsTitles]
    
    return ma_df




In [4]:
#ma_df = get_ma_df(df, 14, 50)

In [5]:
#ma_df


### Bollinger Band

In [6]:
def get_bb_df(df, k,win):
    bb_df = df.copy()
    bb_df.rename(columns={'Adj Close': 'Price'}, inplace = True)
    bb_df = bb_df[['Date', 'Price']]
    bb_df['bb_ma'] = bb_df['Price'].rolling(win).mean()
    bb_df['bb_std'] = bb_df['Price'].rolling(win).std()        
    bb_df['lower_band'] = bb_df['bb_ma']- k* bb_df['bb_std']
    bb_df['upper_band'] = bb_df['bb_ma'] + k* bb_df['bb_std']
    bb_df['bb_sell'] = bb_df['Price'] < bb_df['lower_band']
    bb_df['bb_buy'] = bb_df['Price'] > bb_df['upper_band']
        

    columnsTitles = ['bb_buy','bb_sell']
    bb_df = bb_df[columnsTitles]
    
    return bb_df




In [7]:
#bb_df = get_bb_df(df,2,56)
#bb_df

### Consecutive days

In [8]:
def compute_percent_df(df):
    try:
        new_df = df.copy()
        new_df.rename(columns={'Adj Close': 'Price'}, inplace = True)
        new_df = new_df[['Date', 'Price']]
        new_df['price_change(%)'] = new_df['Price'].pct_change() * 100
        new_df.fillna(0, inplace = True)
        return new_df
    except Exception as e:
        print(e)
        return None 


def compute_sell_buy_df(df,w):
    new_df = compute_percent_df(df)
    new_df['good/bad(1/0)'] = np.where(new_df['price_change(%)'] >= 0,1,0)
    
    count = 0
    list = new_df['good/bad(1/0)']
    buy_list = [] # index of row to buy
    sell_list = [] # index or row to sell
    buy_sell_list = []

    for i in range(len(list)):
        if list[i] == 0:
            count +=1
            if count == w:
                count = 0 
                buy_list.append(i)
                sell_list.append(i+1)
                i+=2
            else:
                i+=1
        else:
            count = 0
            i+=1
    
        
    for i in range(len(df)):
        if i in buy_list:
            buy_sell_list.append('buy')
        elif i in sell_list:
            buy_sell_list.append('sell')
        else:
            buy_sell_list.append('hold')
            
    
    new_df['con_buy/sell'] = buy_sell_list
    
    
    columnsTitles = ['con_buy/sell']
    new_df = new_df[columnsTitles]
    
    return new_df



In [9]:
#con_df = compute_sell_buy_df(df,1) 
#con_df

### Combine strategies with ensemble voting as follows: 
### For each day, buy only if at least two other strategies will buy and sell only if at least two other strategies indicate a sell 

In [10]:
def buy(investment,price):
    shares = investment / price
    return shares

def sell(cash,shares, price): # cash = cash in hand (not including stock value)
    revenue = shares * price
    new_cash = revenue - cash
    return new_cash

In [11]:
def get_ens_df(df,win_st,win_lt,k,bb_win,con_win):
    ma_df = get_ma_df(df,win_st,win_lt)
    bb_df = get_bb_df(df, k,bb_win)
    con_df = compute_sell_buy_df(df,con_win)
    
    result_df = pd.concat([ma_df,bb_df, con_df], axis=1)
    ens_buy_sell = []

    for row in range(len(result_df)):
        buy_votes = 0
        sell_votes = 0
        if result_df.loc[row]['ma_buy'] == True:
            buy_votes +=1
        if result_df.loc[row]['bb_buy'] == True:
            buy_votes +=1
        if result_df.loc[row]['con_buy/sell'] == 'buy':
            buy_votes +=1 

        if result_df.loc[row]['ma_sell'] == True:
            sell_votes +=1
        if result_df.loc[row]['bb_sell'] == True:
            sell_votes +=1
        if result_df.loc[row]['con_buy/sell'] == 'sell':
            sell_votes +=1


        if buy_votes >=2:
            ens_buy_sell.append('buy')
        elif sell_votes >=2:
            ens_buy_sell.append('sell')
        else:
            ens_buy_sell.append('hold')

    result_df['ens_buy/sell']= ens_buy_sell
    
    # make sure all the transactions start at buy and end at sell
    transaction_start_index = result_df[(result_df['ens_buy/sell']=='buy')].index.values.astype(int)
    np.array(transaction_start_index).tolist()
    transaction_start_index = transaction_start_index[0]

    transaction_end_index = result_df[(result_df['ens_buy/sell']=='sell')].index.values.astype(int)
    np.array(transaction_end_index).tolist()
    transaction_end_index = transaction_end_index[-1]


    for row in range(len(result_df)):
        if (row < transaction_start_index)|(row > transaction_end_index):
            result_df.at[row,'ens_buy/sell'] = 'hold'
    
    
    # calculate shares and profit


    initial_capital = 100
    cash = []
    shares = []

    for row in range(len(result_df)):
        if row == 0:
            # buy: shares + ; cash = 0
            if result_df.loc[row]['ens_buy/sell']=='buy':
                result_df.at[row,'ens_final_buy/sell'] = 'buy'
                shares.append(buy(initial_capital,result_df.at[row,'Price']))
                cash.append(0)
            # sell : shares - ; cash +/-
            elif result_df.loc[row]['ens_buy/sell']=='sell' :
                result_df.at[row,'ens_final_buy/sell'] = 'hold'
                shares.append(0)
                cash.append(initial_capital)
            # hold : shares and cash no change
            else:
                result_df.at[row,'ens_final_buy/sell'] = 'hold'
                shares.append(0)
                cash.append(initial_capital)

        else:
            # buy: shares + ; cash = 0
            if (result_df.loc[row]['ens_buy/sell']=='buy') & (shares[row-1] == 0):
                result_df.at[row,'ens_final_buy/sell'] = 'buy'
                shares.append(buy(cash[row-1],result_df.at[row,'Price']))
                cash.append(0)
            # sell : shares - ; cash +/-
            elif (result_df.loc[row]['ens_buy/sell']=='sell') & (cash[row-1] == 0):
                result_df.at[row,'ens_final_buy/sell'] = 'sell'
                shares.append(0)
                cash.append(sell(cash[row-1],shares[row-1],result_df.at[row,'Price']))
            # hold : shares and cash no change
            else:
                result_df.at[row,'ens_final_buy/sell'] = 'hold'
                shares.append(shares[row-1])
                cash.append(cash[row-1])

    result_df['ens_shares']= shares
    result_df['ens_cash'] = cash
    
    columnsTitles = ['Date','Price','ens_final_buy/sell','ens_shares','ens_cash']
    result_df = result_df.reindex(columns=columnsTitles)

    return result_df

In [12]:
ens_df = get_ens_df(df,14,50,2,56,3)

In [13]:
ens_df

,Date,Price,ens_final_buy/sell,ens_shares,ens_cash
0,2013/9/16,34.026104,hold,0.000000,100.000000
1,2013/9/17,33.885891,hold,0.000000,100.000000
2,2013/9/18,34.876091,hold,0.000000,100.000000
3,2013/9/19,34.692081,hold,0.000000,100.000000
4,2013/9/20,34.087433,hold,0.000000,100.000000
5,2013/9/23,34.104969,hold,0.000000,100.000000
6,2013/9/24,34.140015,hold,0.000000,100.000000
7,2013/9/25,34.135738,hold,0.000000,100.000000
8,2013/9/26,34.514431,hold,0.000000,100.000000
9,2013/9/27,34.100506,hold,0.000000,100.000000


In [14]:
ma_windows_short = [28,28,14,14,13]
ma_windows_long = [63,56,70,63,56]
bb_k = [3,2.5,2.75,2.5,2]
bb_w = [56,84,63,98,77]
con_w = [2,3,4]

ens_P_L = []


for i in range(5):
    for j in range(5):
        for k in range(3):
            ens_df = get_ens_df(df,ma_windows_short[i],ma_windows_long[j],bb_k[i],bb_w[j],con_w[k])
            ens_profit = ens_df.loc[len(ens_df)-1]['ens_cash'] 
            ens_P_L.append(ens_profit)

            



            
d = {'ens_P_L': ens_P_L}
df_result = pd.DataFrame(data=d)                  
df_result
            

,ens_P_L
0,84.182512
1,98.191075
2,97.743029
3,97.241429
4,102.840916
5,102.343376
6,81.185139
7,96.936933
8,98.208452
9,86.049817


In [15]:
df_result.to_csv('ens.csv')